In [177]:
#import dependencies 
from numpy.random import seed
from numpy.random import randn
from scipy import stats
import numpy as np

import math  

import seaborn as sns
sns.set(style="darkgrid")
from matplotlib import pyplot as plt

import pandas as pd
import os

In [508]:
#check list of files in data directory 
data_folder = './data/dedicated_host/11-16-2019//'
print(os.listdir(data_folder))

['merged_new.csv', '.DS_Store', 'm5d_pgbench_dedicated_host_11-16-2019.csv', 'm5d_y-cruncher_dedicated_host_11-16-2019_us-east-1 - Sheet1.csv', 'm5d_iperf_dedicated_host_11-17.csv', 'm5d_sysbench_dedicated_host_11-16-2019_us-east-1b - Sheet1.csv']


In [552]:
#make sure to import correct file here
data_name = 'm5d_sysbench_dedicated_host_11-16-2019_us-east-1b - Sheet1.csv'

In [553]:
#adjust header if necessary
df_data = pd.read_csv(data_folder + data_name, header = 1)

In [554]:
#remove idle vm rows

#y-cruncher
#df_data.dropna(subset=['processorInfo'], inplace= True)

In [555]:
df_data.head()

,experimentID,instanceID,instanceType,wallTime,testOption,per-request-avg-time,total-time,thread-num
0,1573960552-0,i-0d6feb91baa545931,m5d.large,9.157414,--test=cpu --cpu-max-prime=2000000 --num-threa...,1802.76ms,9.0154s,2
1,1573960552-0,i-0d6feb91baa545931,m5d.large,9.038783,--test=cpu --cpu-max-prime=2000000 --num-threa...,1803.40ms,9.0174s,2
2,1573960552-0,i-0d6feb91baa545931,m5d.large,9.039020,--test=cpu --cpu-max-prime=2000000 --num-threa...,1802.86ms,9.0164s,2
3,1573960552-0,i-0d6feb91baa545931,m5d.large,9.041733,--test=cpu --cpu-max-prime=2000000 --num-threa...,1802.86ms,9.0184s,2
4,1573960552-0,i-0d6feb91baa545931,m5d.large,9.047082,--test=cpu --cpu-max-prime=2000000 --num-threa...,1803.25ms,9.0220s,2


In [556]:
#setup cols, metric for the relative experiment type
def setUpMetrics(experimentName):
    
    if experimentName == 'pgbench':
        cols = ['instanceID', 'setId', 'transactions']
        metric = 'transactions'
    elif experimentName == 'sysbench' or experimentName == 'y-cruncher':
        cols = ['instanceID','setId','wallTime']
        metric = 'wallTime'
    elif experimentName == 'iperf':
        df_data['Total'] = df_data['Upload'] + df_data['Download']
        cols = ['instanceID','setId','Upload', 'Download', 'Total']
        metric = 'Total'
    else:
        print('Illegal Experiment Name..')
    return metric, cols

In [557]:
#setup vars for each experiment time... i.e. wallTime vs transactions
experimentName = 'sysbench'
metric, cols = setUpMetrics(experimentName)

In [558]:
#create setId column
df_data['setId'] = df_data['experimentID'].str.strip().str[-2:]
df_data['setId'] = df_data['setId'].astype('int32').abs()

In [559]:
df_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23520 entries, 0 to 23519
Data columns (total 9 columns):
experimentID            23520 non-null object
instanceID              23520 non-null object
instanceType            23520 non-null object
wallTime                23520 non-null float64
testOption              23520 non-null object
per-request-avg-time    23520 non-null object
total-time              23520 non-null object
thread-num              23520 non-null int64
setId                   23520 non-null int32
dtypes: float64(1), int32(1), int64(1), object(6)
memory usage: 1.5+ MB


In [560]:
df_data.tail()

,experimentID,instanceID,instanceType,wallTime,testOption,per-request-avg-time,total-time,thread-num,setId
23515,1573960552-23,i-07998976f194caa97,m5d.large,8.010195,--test=cpu --cpu-max-prime=2000000 --num-threa...,1597.11ms,7.9879s,2,23
23516,1573960552-23,i-07998976f194caa97,m5d.large,8.009076,--test=cpu --cpu-max-prime=2000000 --num-threa...,1596.62ms,7.9858s,2,23
23517,1573960552-23,i-07998976f194caa97,m5d.large,8.017726,--test=cpu --cpu-max-prime=2000000 --num-threa...,1598.08ms,7.9976s,2,23
23518,1573960552-23,i-07998976f194caa97,m5d.large,8.005946,--test=cpu --cpu-max-prime=2000000 --num-threa...,1596.45ms,7.9857s,2,23
23519,1573960552-23,i-07998976f194caa97,m5d.large,8.008423,--test=cpu --cpu-max-prime=2000000 --num-threa...,1597.13ms,7.9881s,2,23


In [561]:
#drop other columns
df_data = df_data[cols]

In [562]:
df_data.head(13)

,instanceID,setId,wallTime
0,i-0d6feb91baa545931,0,9.157414
1,i-0d6feb91baa545931,0,9.038783
2,i-0d6feb91baa545931,0,9.039020
3,i-0d6feb91baa545931,0,9.041733
4,i-0d6feb91baa545931,0,9.047082
5,i-0d6feb91baa545931,0,9.060294
6,i-0d6feb91baa545931,0,9.040257
7,i-0d6feb91baa545931,0,9.037618
8,i-0d6feb91baa545931,0,9.043960
9,i-0d6feb91baa545931,0,9.037151


In [563]:
df_data.describe()

,setId,wallTime
count,23520.000000,23520.000000
mean,15.666667,8.427547
std,11.426334,0.546344
min,0.000000,7.169777
25%,6.000000,7.786108
50%,14.000000,8.772248
75%,24.000000,8.797785
max,47.000000,10.932398


In [564]:
df_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23520 entries, 0 to 23519
Data columns (total 3 columns):
instanceID    23520 non-null object
setId         23520 non-null int32
wallTime      23520 non-null float64
dtypes: float64(1), int32(1), object(1)
memory usage: 459.5+ KB


In [565]:
#throw out every nth_value
nth_column = 10
df_data = df_data[df_data.index % nth_column != 0]

In [581]:
df_data.head(12)

,instanceID,setId,wallTime
1,i-0d6feb91baa545931,0,9.038783
2,i-0d6feb91baa545931,0,9.039020
3,i-0d6feb91baa545931,0,9.041733
4,i-0d6feb91baa545931,0,9.047082
5,i-0d6feb91baa545931,0,9.060294
6,i-0d6feb91baa545931,0,9.040257
7,i-0d6feb91baa545931,0,9.037618
8,i-0d6feb91baa545931,0,9.043960
9,i-0d6feb91baa545931,0,9.037151
11,i-0d6feb91baa545931,1,9.049554


In [567]:
#group by tenants / setId 
mean = df_data.groupby('setId').mean()
mean.head()

,wallTime
setId,
0,9.053729
1,9.050629
2,9.048362
3,9.051012
4,8.927001


In [568]:
#create result dataframe
result_df = mean

In [569]:
#create std, normalized data
std = df_data.groupby('setId').std()
std.head()

,wallTime
setId,
0,0.119393
1,0.017545
2,0.012864
3,0.016248
4,0.130973


In [570]:
#add normalized column
if experimentName == 'pgbench':
    result_df['normalized'] = (result_df.loc[:,metric:]-result_df.iloc[-1][metric:]).div(result_df.iloc[-1][metric:])
    result_df['normalized'] *= 100
    result_df['normalized'] += 100
else: 
    result_df['normalized'] = 1 - (result_df.loc[:,metric:]-result_df.iloc[-1][metric:]).div(result_df.iloc[-1][metric:])
    result_df['normalized'] *= 100

result_df['normalized'] = result_df['normalized'].round(2).astype(str) + '%'

In [571]:
result_df.head()

,wallTime,normalized
setId,,
0,9.053729,73.8%
1,9.050629,73.84%
2,9.048362,73.87%
3,9.051012,73.83%
4,8.927001,75.56%


In [572]:
#add std column
result_df['std'] = std

In [573]:
#reverse table
result_df=result_df.iloc[::-1]

In [574]:
#add tenants
result_df['tenants'] = np.arange(len(result_df))+1

/Users/dell/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [575]:
#rearrange columns
cols = ['tenants', metric, 'std', 'normalized']

In [576]:
result_df = result_df[cols]

In [577]:
#add % to normalized

In [578]:
result_df

,tenants,wallTime,std,normalized
setId,,,,
47,1,7.173863,0.002781,100.0%
46,2,7.176439,0.004802,99.96%
45,3,7.568770,0.004558,94.5%
44,4,7.568645,0.003861,94.5%
43,5,7.781149,0.004479,91.53%
42,6,7.813378,0.123144,91.09%
41,7,7.780429,0.004467,91.54%
40,8,7.781238,0.005390,91.53%
39,9,7.713974,0.190197,92.47%


In [579]:
#print to csv
result_df.to_csv( path_or_buf='./data/summary/' + data_name + '-Summary.csv')

In [580]:
# graph tenants vs wallTime